In [ ]:
import os
import sys
import math
import argparse
import numpy as np
from collections import Counter
import xml.etree.ElementTree as et 

# Keras imports
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils.vis_utils import model_to_dot
from tensorflow.keras.layers import Input, Dense, Dropout, AlphaDropout, BatchNormalization, Activation

# ml4h Imports
from ml4h.plots import plot_ecg
from ml4h.arguments import parse_args
from ml4h.recipes import train_multimodal_multitask

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
strip_ekgs = {}
ffts = {}
xml_file = '/mnt/disks/data/raw/ecgs/5223300_20205_2_0.xml'

root = et.parse(xml_file).getroot()
for c in root.findall("./StripData/WaveformData"):
    strip_ekgs[c.attrib['lead']] = np.array(list(map(float, c.text.strip().split(','))))

plot_ecg(strip_ekgs, 'strip')

In [ ]:
for c in root.findall("./RestingECGMeasurements"):
    for child in c:
        print(child.tag, child.text)
        
for d in root.findall("./Interpretation/Diagnosis/DiagnosisText"):
	diagnosis = ''.join(e for e in d.text if e.isalnum() or e == ' ')
	print(diagnosis)

rows = int(math.ceil(math.sqrt(len(strip_ekgs))))
cols = math.ceil(len(strip_ekgs)/rows)
fig, axes = plt.subplots(rows, cols, figsize=(28,24))
ffts = {}
filtered = {}
start_filter = 0
end_filter = 100
band_pass = True  # otherwise band block
for i,k in enumerate(strip_ekgs):
    ax = plt.subplot(rows, cols, i+1)
    ax.set_title('Frequency Magnitude '+k)
    ffts[k] = np.fft.fft(strip_ekgs[k])
    freq = np.fft.fftfreq(ffts[k].shape[-1])
    if band_pass:
        ffts[k][:start_filter] = np.zeros((start_filter,))
        ffts[k][end_filter:] = np.zeros((len(ffts[k])-end_filter,))
    else:
        ffts[k][start_filter:end_filter] = np.zeros(np.abs(end_filter-start_filter,))  
    filtered[k] = np.fft.ifft(ffts[k])
    ax.plot(freq, np.abs(ffts[k]))
    ax.set_ylim([-20, 60000])
plt.tight_layout()
plt.show()

In [ ]:

rows = int(math.ceil(math.sqrt(len(ffts))))
cols = math.ceil(len(ffts)/rows)
fig, axes = plt.subplots(rows, cols, figsize=(28,24))
for i,k in enumerate(filtered):
    ax = plt.subplot(rows, cols, i+1)
    ax.set_title('Filtered '+k)
    ax.plot(np.abs(filtered[k]))
    #ax.set_ylim([-20, 60000])
    #ax.set_xlim([-0.06, 0.06])
plt.tight_layout()
plt.show()    

In [ ]:
xml_file = '/mnt/disks/data/raw/ecgs/4856206_20205_2_0.xml'

root = et.parse(xml_file).getroot()
for c in root.findall("./StripData/WaveformData"):
    strip_ekgs[c.attrib['lead']] = np.array(list(map(float, c.text.strip().split(','))))
    ffts[c.attrib['lead']] = np.fft.fft(strip_ekgs[c.attrib['lead']])

plot_ecg(strip_ekgs, 'strip_bradycardia')

In [ ]:
root = et.parse(xml_file).getroot()
for d in root.findall("./Interpretation/Diagnosis/DiagnosisText"):
    diagnosis = ''.join(e for e in d.text if e.isalnum() or e == ' ')
    print(diagnosis)

rows = int(math.ceil(math.sqrt(len(ffts))))
cols = math.ceil(len(ffts)/rows)
fig, axes = plt.subplots(rows, cols, figsize=(28,24))

for i,k in enumerate(ffts):
    ax = plt.subplot(rows, cols, i+1)
    ax.set_title('FFT Magnitude '+k)
    freq = np.fft.fftfreq(ffts[k].shape[-1])
    ax.plot(freq, np.abs(ffts[k]))
    ax.set_ylim([-20, 80000])
    ax.set_xlim([-0.08, 0.08])
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(rows, cols, figsize=(28,24))
for i,k in enumerate(ffts):
    ax = plt.subplot(rows, cols, i+1)
    ax.set_title('FFT Magnitude '+k)
    freq = np.fft.fftfreq(ffts[k].shape[-1])
    ax.plot(freq, ffts[k].real, freq, ffts[k].imag)
    ax.set_ylim([-80000, 80000])
    ax.set_xlim([-0.04, 0.04])
plt.tight_layout()
plt.show()